In [59]:
import os
import torch
import sys
sys.path.append("../code")
sys.path.append("../pytorch_resnet_preact")
from torch.utils.tensorboard import SummaryWriter
import Model, Network, utils, ImageUtils, DataLoader, Configure
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
# Aug1= ImageUtils.ImgAugTransformStandard
# Aug2 = ImageUtils.ImgAugTransform1
# Aug3 = ImageUtils.ImgAugTransform2
Aug1 = ImageUtils.ImgTransformStandard
Aug2 = ImageUtils.ImgTransform1
Aug3 = ImageUtils.ImgTransform2
Aug4 = ImageUtils.ImgTransform3

train_1,_,_ = DataLoader.load_data('../data',train_aug=Aug1)
train_2,_,_ = DataLoader.load_data('../data',train_aug=Aug2)
train_3,_,_ = DataLoader.load_data('../data',train_aug=Aug3)
train_4,_,_ = DataLoader.load_data('../data',train_aug=Aug4)

train_1 = torch.utils.data.DataLoader(train_1,100,shuffle=False)
train_2 = torch.utils.data.DataLoader(train_2,100,shuffle=False)
train_3 = torch.utils.data.DataLoader(train_3,100,shuffle=False)
train_4 = torch.utils.data.DataLoader(train_4,100,shuffle=False)


invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.2470, 1/0.2435, 1/0.2616 ]),
                                transforms.Normalize(mean = [ -0.4914, -0.4822, -0.4465 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [54]:
## Generate the plot images for transforms

i = 0
for ti in [train_1,train_2,train_3,train_4]:
    i+=1
    for batch_idx, (inputs, targets) in enumerate(ti):
        #print("Shape of Training Images after TorchTransform")
        print(inputs.shape)
        if batch_idx >= 0:
            break

    images = inputs[0:12]
    fig = plt.figure(figsize=(12, 3))
    for idx in np.arange(12):
        
        image = invTrans(images[idx]).cpu().numpy()
        image[image < 0] = 0.0
        ax = fig.add_subplot(2, 6, idx+1, xticks=[], yticks=[])
        plt.imshow(np.transpose(image.astype(np.float), (1, 2, 0)),interpolation='nearest', aspect='equal')
    plt.show()
    plt.savefig(str(i)+'.png')

torch.Size([30, 3, 32, 32])
torch.Size([30, 3, 32, 32])
torch.Size([30, 3, 32, 32])
torch.Size([30, 3, 32, 32])


In [62]:
import time
import tqdm
### Test the computational power needed for transforms:
times = np.zeros((4,len(train_1)))

train_iter = iter(train_1)
    

idx = 0
pbar = tqdm.tqdm(total=4*len(train_1))

for ti in [train_1,train_2,train_3,train_4]:
    train_iter = iter(ti)
    for i in range(len(train_1)):
        t0 = time.time()
        _,_ = next(train_iter)
        times[idx,i] = time.time() - t0
        pbar.update(1)
    idx +=1


100%|█████████▉| 1998/2000 [01:11<00:00, 33.68it/s]

In [75]:
import matplotlib
matplotlib.use('tkagg') # command-line use only
matplotlib.style.use('ggplot')

plt.figure()

plt.hist(times[0]*100,label='Transform Standard',alpha=0.5)
plt.hist(times[1]*100,label='Transform1',alpha=0.5)
plt.hist(times[2]*100,label='Transform2',alpha=0.5)
plt.hist(times[3]*100,label='Transform3',alpha=0.5)
plt.xlabel('Execution Time [ms]')
plt.ylabel('Freq')
plt.title('Execution Performance for Image Augmentations')
plt.legend(loc='best')
plt.show()